<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_5_3___multimodal_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -qU llama-index-callbacks-arize-phoenix llama-parse llama-index-llms-openai llama-index-embeddings-openai llama-index-multi-modal-llms-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB

Ten kod instaluje kilka pakietów Pythona związanych z LlamaIndex - narzędziem do pracy z dużymi modelami językowymi i danymi:

`llama-index-callbacks-arize-phoenix` - moduł do integracji z platformą Arize Phoenix, która służy do monitorowania i analizowania modeli ML

`llama-parse` - parser dokumentów, który przekształca różne formaty (PDF, HTML itp.) na ustrukturyzowane dane

`llama-index-llms-openai` - adapter do wykorzystania modeli językowych OpenAI w LlamaIndex

`llama-index-embeddings-openai` - moduł do tworzenia embedingów (wektorowych reprezentacji tekstu) przy użyciu API OpenAI

`llama-index-multi-modal-llms-openai` - rozszerzenie umożliwiające pracę z modelami multimodalnymi OpenAI, które potrafią przetwarzać zarówno tekst jak i obrazy

Flaga `-q` wycisza standardowe komunikaty instalacyjne, a `-U` wymusza aktualizację do najnowszych wersji pakietów.

In [ ]:
# Standard library imports
import os
import re
from copy import deepcopy
from pathlib import Path
from typing import Optional
from google.colab import userdata
from pprint import pp as ppr

# Third-party imports
from llama_parse import LlamaParse

# LlamaIndex core imports
from llama_index.core import Settings, StorageContext, VectorStoreIndex, load_index_from_storage
from llama_index.core.schema import TextNode, ImageNode, NodeWithScore, MetadataMode
from llama_index.core.query_engine import CustomQueryEngine, SimpleMultiModalQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.prompts import PromptTemplate
from llama_index.core.base.response.schema import Response
from llama_index.core.tools import QueryEngineTool
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core import set_global_handler

# LlamaIndex service-specific imports
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.multi_modal_llms.openai import OpenAIMultiModal

import nest_asyncio
nest_asyncio.apply()

Ten kod zawiera importy niezbędnych bibliotek i modułów:

Ze standardowej biblioteki Pythona:
- `os` - operacje na systemie operacyjnym i ścieżkach
- `re` - obsługa wyrażeń regularnych
- `deepcopy` - tworzenie głębokich kopii obiektów
- `Path` - obiektowa obsługa ścieżek do plików
- `Optional` - typ dla wartości opcjonalnych
- `userdata` - dostęp do danych użytkownika w Google Colab
- `ppr` - funkcja do ładnego wyświetlania struktur danych

Z zewnętrznych bibliotek:
- `LlamaParse` - narzędzie do parsowania dokumentów

Z głównego modułu LlamaIndex importowane są podstawowe komponenty:
- `Settings`, `StorageContext` - konfiguracja i zarządzanie przechowywaniem
- `VectorStoreIndex` - indeks wektorowy dokumentów
- `load_index_from_storage` - wczytywanie zapisanego indeksu
- Klasy do obsługi różnych typów węzłów (`TextNode`, `ImageNode`)
- Komponenty do budowy silników zapytań i ich obsługi
- System promptów i odpowiedzi
- Narzędzia do tworzenia agentów

Ze specyficznych modułów LlamaIndex:
- Integracja z OpenAI (modele językowe, embedingi, modele multimodalne)

Na końcu włączany jest `nest_asyncio`, który umożliwia zagnieżdżone pętle asynchroniczne w Jupyterze/Colab.

In [ ]:
class CFG:
  model1 = "gpt-4o"
  model2 = "text-embedding-3-large"
  temperature = 0.1
  chunksize = 1024

In [ ]:
# trace the whole thing
PHOENIX_API_KEY = userdata.get('arize')
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)


# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = userdata.get('openaivision')

# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get('llamaparse')

Ten fragment kodu konfiguruje śledzenie (tracing) i klucze API dla różnych usług:

1. Konfiguracja Arize Phoenix do śledzenia:
- Pobiera klucz API Arize z danych użytkownika
- Ustawia go jako nagłówek dla eksportera OTEL (OpenTelemetry)
- Konfiguruje globalny handler do wysyłania śladów na llamatrace.com

2. Ustawia klucz API OpenAI:
- Pobiera klucz dostępu do OpenAI Vision z danych użytkownika
- Zapisuje go jako zmienną środowiskową OPENAI_API_KEY
- Ten klucz będzie używany do embedingów i modeli językowych

3. Konfiguruje dostęp do Llama Cloud:
- Pobiera klucz API dla usługi llama-parse
- Zapisuje go jako zmienną środowiskową LLAMA_CLOUD_API_KEY

Wszystkie klucze są pobierane bezpiecznie z danych użytkownika w Google Colab zamiast być zapisane bezpośrednio w kodzie. Te ustawienia umożliwiają monitorowanie działania aplikacji oraz dostęp do zewnętrznych API potrzebnych do przetwarzania dokumentów i generowania odpowiedzi.

In [ ]:

llm = OpenAI(model= CFG.model1)
Settings.llm = llm

embed_model = OpenAIEmbedding(model = CFG.model2)
Settings.embed_model = embed_model


Ten kod konfiguruje dwa kluczowe komponenty w LlamaIndex:

1. Model językowy (LLM):
- Tworzy instancję modelu OpenAI, używając wartości `model1` z konfiguracji
- Ustawia ten model jako globalny LLM w ustawieniach

2. Model do tworzenia embedingów:
- Tworzy instancję modelu OpenAIEmbedding, używając wartości `model2` z konfiguracji
- Ustawia ten model jako globalny model embedingów w ustawieniach

Te globalne ustawienia sprawiają, że wszystkie komponenty LlamaIndex będą domyślnie używać tych samych modeli, zapewniając spójność w całej aplikacji. Embedingi służą do przekształcania tekstu na wektory liczbowe, co umożliwia wyszukiwanie semantyczne, a model językowy jest używany do generowania odpowiedzi.

# Functions

In [ ]:
def get_page_number(file_name):
    match = re.search(r"-page-(\d+)\.jpg$", str(file_name))
    if match:
        return int(match.group(1))
    return 0


Ta funkcja wydobywa numer strony z nazwy pliku:

- Przyjmuje parametr `file_name` (nazwa pliku)
- Używa wyrażenia regularnego do znalezienia wzorca "-page-X.jpg", gdzie X to liczba
- `r"-page-(\d+)\.jpg$"` to wzorzec, który:
  - Szuka tekstu "-page-"
  - Następnie szuka jednej lub więcej cyfr (`\d+`)
  - Kończy się na ".jpg" (`\.jpg$`)
- Jeśli wzorzec zostanie znaleziony, funkcja zwraca znalezioną liczbę jako integer
- Jeśli wzorzec nie zostanie znaleziony, zwraca 0


In [ ]:
def _get_sorted_image_files(image_dir):
    """Get image files sorted by page."""
    raw_files = [f for f in list(Path(image_dir).iterdir()) if f.is_file()]
    sorted_files = sorted(raw_files, key=get_page_number)
    return sorted_files


Ta funkcja sortuje pliki obrazów według numeru strony:

- Przyjmuje parametr `image_dir` - ścieżkę do katalogu z obrazami
- Używa `Path(image_dir).iterdir()` do iteracji po zawartości katalogu
- Tworzy listę zawierającą tylko pliki (pomija podkatalogi) poprzez `is_file()`
- Sortuje listę plików używając wcześniej zdefiniowanej funkcji `get_page_number` jako klucza sortowania
- Zwraca posortowaną listę plików


In [ ]:
# attach image metadata to the text nodes
def get_text_nodes(docs, image_dir=None, json_dicts=None):
    """Split docs into nodes, by separator."""
    nodes = []

    image_files = _get_sorted_image_files(image_dir) if image_dir is not None else None
    md_texts = [d["md"] for d in json_dicts] if json_dicts is not None else None

    doc_chunks = [c for d in docs for c in d.text.split("---")]
    for idx, doc_chunk in enumerate(doc_chunks):
        chunk_metadata = {"page_num": idx + 1}
        if image_files is not None:
            image_file = image_files[idx]
            chunk_metadata["image_path"] = str(image_file)
        if md_texts is not None:
            chunk_metadata["parsed_text_markdown"] = md_texts[idx]
        chunk_metadata["parsed_text"] = doc_chunk
        node = TextNode(
            text="",
            metadata=chunk_metadata,
        )
        nodes.append(node)

    return nodes

Ta funkcja przetwarza dokumenty na węzły tekstu z metadanymi:

- Przyjmuje parametry:
  - `docs` - dokumenty do przetworzenia
  - `image_dir` - opcjonalny katalog z obrazami
  - `json_dicts` - opcjonalny słownik z tekstami w formacie Markdown

- Działanie:
  - Jeśli podano `image_dir`, pobiera posortowane pliki obrazów
  - Jeśli podano `json_dicts`, wyciąga teksty Markdown
  - Dzieli każdy dokument na części używając separatora "---"
  - Dla każdej części tworzy metadane zawierające:
    - Numer strony
    - Ścieżkę do obrazu (jeśli dostępna)
    - Tekst w formacie Markdown (jeśli dostępny)
    - Oryginalny tekst części
  - Tworzy węzeł `TextNode` z pustym tekstem i metadanymi
  - Dodaje węzeł do listy

- Zwraca listę utworzonych węzłów

Funkcja służy do łączenia tekstu z odpowiadającymi mu obrazami i dodatkowymi metadanymi, zachowując informację o numeracji stron.

In [ ]:
def get_nodes(docs):
    """Split docs into nodes, by separator."""
    nodes = []
    for doc in docs:
        doc_chunks = doc.text.split("\n---\n")
        for doc_chunk in doc_chunks:
            node = TextNode(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata),
            )
            nodes.append(node)

    return nodes

Ta funkcja przetwarza dokumenty na węzły tekstu:

- Przyjmuje listę dokumentów (`docs`)
- Dla każdego dokumentu:
  - Dzieli jego tekst na części używając separatora "\n---\n"
  - Dla każdej części tworzy nowy węzeł `TextNode` zawierający:
    - Tekst danej części
    - Głęboką kopię metadanych oryginalnego dokumentu (używając `deepcopy`)
  - Dodaje utworzony węzeł do listy
- Zwraca listę wszystkich utworzonych węzłów

W przeciwieństwie do poprzedniej funkcji `get_text_nodes`, ta wersja:
- Używa tekstu bezpośrednio w węźle zamiast w metadanych
- Zachowuje oryginalne metadane dokumentu
- Ma prostszą strukturę bez obsługi obrazów i Markdown
- Używa innego separatora ("\n---\n" zamiast "---")

In [ ]:

class MultimodalQueryEngine(CustomQueryEngine):
    """Custom multimodal Query Engine.

    Takes in a retriever to retrieve a set of document nodes.
    Also takes in a prompt template and multimodal model.

    """

    qa_prompt: PromptTemplate
    retriever: BaseRetriever
    multi_modal_llm: OpenAIMultiModal

    def __init__(self, qa_prompt: Optional[PromptTemplate] = None, **kwargs) -> None:
        """Initialize."""
        super().__init__(qa_prompt=qa_prompt or QA_PROMPT, **kwargs)

    def custom_query(self, query_str: str):
        # retrieve text nodes
        nodes = self.retriever.retrieve(query_str)
        # create ImageNode items from text nodes
        image_nodes = [
            NodeWithScore(node=ImageNode(image_path=n.metadata["image_path"]))
            for n in nodes
        ]

        # create context string from text nodes, dump into the prompt
        context_str = "\n\n".join(
            [r.get_content(metadata_mode=MetadataMode.LLM) for r in nodes]
        )
        fmt_prompt = self.qa_prompt.format(context_str=context_str, query_str=query_str)

        # synthesize an answer from formatted text and images
        llm_response = self.multi_modal_llm.complete(
            prompt=fmt_prompt,
            image_documents=[image_node.node for image_node in image_nodes],
        )
        return Response(
            response=str(llm_response),
            source_nodes=nodes,
            metadata={"text_nodes": text_nodes, "image_nodes": image_nodes},
        )

#         return response

Ta klasa implementuje silnik zapytań obsługujący dane multimodalne (tekst i obrazy):

Główne komponenty:
- `qa_prompt` - szablon promptu do zadawania pytań
- `retriever` - moduł do wyszukiwania odpowiednich dokumentów
- `multi_modal_llm` - model językowy obsługujący zarówno tekst jak i obrazy

Metoda `custom_query`:
- Przyjmuje zapytanie tekstowe
- Wyszukuje odpowiednie węzły tekstowe
- Tworzy węzły obrazów na podstawie ścieżek zapisanych w metadanych węzłów tekstowych
- Łączy zawartość tekstową węzłów w jeden kontekst
- Formatuje prompt używając kontekstu i zapytania
- Generuje odpowiedź używając modelu multimodalnego, który analizuje zarówno tekst jak i obrazy
- Zwraca odpowiedź w formie obiektu `Response` zawierającego:
  - Wygenerowaną odpowiedź tekstową
  - Użyte węzły źródłowe
  - Metadane z użytymi węzłami tekstowymi i obrazowymi


# Data

In [ ]:
!mkdir data
!mkdir data_images
!wget "https://static.conocophillips.com/files/2023-conocophillips-aim-presentation.pdf" -O data/conocophillips.pdf

--2024-11-29 08:15:50--  https://static.conocophillips.com/files/2023-conocophillips-aim-presentation.pdf
Resolving static.conocophillips.com (static.conocophillips.com)... 13.249.126.4, 13.249.126.43, 13.249.126.99, ...
Connecting to static.conocophillips.com (static.conocophillips.com)|13.249.126.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41895745 (40M) [application/pdf]
Saving to: ‘data/conocophillips.pdf’

data/conocophillips 100%[===================>]  39.95M  38.8MB/s    in 1.0s    

2024-11-29 08:15:52 (38.8 MB/s) - ‘data/conocophillips.pdf’ saved [41895745/41895745]



# Parse text / img

In [ ]:

parser_text = LlamaParse(result_type="text")
parser_gpt4o = LlamaParse(result_type="markdown", gpt4o_mode=True)


Ten kod tworzy dwie instancje parsera LlamaParse:

`parser_text`:
- Ustawiony na zwracanie wyniku jako czysty tekst
- Nie używa specjalnego trybu GPT-4
- Służy do podstawowego parsowania dokumentów

`parser_gpt4o`:
- Ustawiony na zwracanie wyniku w formacie Markdown
- Ma włączony specjalny tryb `gpt4o_mode`
- Będzie generował bardziej ustrukturyzowany wynik, prawdopodobnie zoptymalizowany pod kątem współpracy z GPT-4

In [ ]:
print(f"Parsing text...")
docs_text = parser_text.load_data("data/conocophillips.pdf")
print(f"Parsing PDF file...")
md_json_objs = parser_gpt4o.get_json_result("data/conocophillips.pdf")
md_json_list = md_json_objs[0]["pages"]

Parsing text...
Started parsing the file under job_id 6dcc9859-d381-446f-b3b1-ec428af777cc
Parsing PDF file...
Started parsing the file under job_id affd3516-6466-4de8-a372-fbc4f7d6342b


Ten kod parsuje plik PDF na dwa różne sposoby:

1. `docs_text = parser_text.load_data("data/conocophillips.pdf")`
- Wczytuje PDF używając pierwszego parsera (text)
- Tworzy prostą reprezentację tekstową dokumentu
- Zapisuje wynik w `docs_text`

2. `md_json_objs = parser_gpt4o.get_json_result("data/conocophillips.pdf")`
- Parsuje ten sam PDF używając drugiego parsera (markdown + gpt4o)
- Zwraca wynik w formacie JSON
- `md_json_list = md_json_objs[0]["pages"]` wyciąga listę stron z pierwszego obiektu w wynikach

In [ ]:
print(md_json_list[2]["md"])

# Cautionary Statement

This presentation provides management’s current operational plan for ConocoPhillips over roughly the next decade, for the assets currently in our portfolio, and is subject to multiple assumptions, including, unless otherwise specifically noted:

- an oil price of $60/BBL West Texas Intermediate in 2022 dollars, escalating at 2.25% annually;
- an oil price of $65/BBL Brent in 2022 dollars, escalating at 2.25% annually;
- a gas price of $3.75/MMBTU Henry Hub in 2022 dollars, escalating at 2.25% annually;
- an international gas price of $8/MMBTU Title Transfer Facility & Japan Korea Marker in 2022 dollars, escalating at 2.25% annually;
- cost and capital escalation in line with price escalation; planning case of $60/BBL WTI assumes capital de-escalation from levels observed in 2022;
- all production compound annual growth rates (CAGR) are calculated for the 10-year period 2023 – 2032;
- inclusion of carbon tax in the cash flow forecasts for assets where a tax is cu

In [ ]:
image_dicts = parser_gpt4o.get_images(md_json_objs, download_path="data_images")


> Image for page 1: [{'name': 'page_1.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 2: [{'name': 'page_2.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 3: [{'name': 'page_3.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 4: [{'name': 'page_4.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 5: [{'name': 'page_5.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 6: [{'name': 'page_6.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 7: [{'name': 'page_7.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 8: [{'name': 'page_8.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}]
> Image for page 9: [{'name': 'page_9.jpg', 'height': 0,

Ta linia kodu wyodrębnia obrazy z przetworzonego dokumentu PDF:

`parser_gpt4o.get_images(md_json_objs, download_path="data_images")`
- Używa parsera w trybie GPT-4 do wyciągnięcia obrazów
- Przyjmuje wcześniej utworzone obiekty JSON (`md_json_objs`)
- Zapisuje znalezione obrazy w katalogu "data_images"
- Zwraca słownik z informacjami o wyodrębnionych obrazach, który zostaje zapisany w zmiennej `image_dicts`

Funkcja ta umożliwia zachowanie zarówno tekstu jak i obrazów z oryginalnego PDF-a, co jest kluczowe dla późniejszego przetwarzania multimodalnego.

# Multimodal index

In [ ]:
text_nodes = get_text_nodes(docs_text, image_dir="data_images", json_dicts=md_json_list)

Ta linia tworzy węzły tekstowe łącząc trzy źródła danych:

`docs_text` - zawartość tekstowa z PDF-a
`image_dir="data_images"` - katalog z wyodrębnionymi obrazami
`json_dicts=md_json_list` - lista stron w formacie Markdown

Funkcja `get_text_nodes`:
- Łączy tekst, obrazy i znaczniki Markdown w węzły
- Dla każdej części dokumentu tworzy węzeł zawierający:
  - Numer strony
  - Ścieżkę do odpowiadającego obrazu strony
  - Tekst w formacie Markdown
  - Oryginalny tekst
- Zachowuje odpowiednią kolejność stron

Wynik jest zapisywany w zmiennej `text_nodes`, która będzie używana do dalszego przetwarzania i wyszukiwania informacji.

In [ ]:

if not os.path.exists("storage_nodes"):
    index = VectorStoreIndex(text_nodes, embed_model=embed_model)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage_nodes")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage_nodes")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

retriever = index.as_retriever()

Ten kod zarządza indeksem wektorowym dokumentów:

Jeśli katalog "storage_nodes" nie istnieje:
- Tworzy nowy indeks wektorowy z węzłów tekstowych
- Używa wcześniej skonfigurowanego modelu do embedingów
- Ustawia identyfikator indeksu jako "vector_index"
- Zapisuje indeks na dysku w katalogu "./storage_nodes"

Jeśli katalog istnieje:
- Odtwarza kontekst przechowywania z istniejącego katalogu
- Wczytuje zapisany indeks o identyfikatorze "vector_index"

Na końcu tworzy obiekt `retriever` z indeksu, który będzie służył do wyszukiwania odpowiednich fragmentów dokumentów podczas odpowiadania na zapytania.

Takie podejście pozwala zaoszczędzić czas i zasoby - raz utworzony indeks jest przechowywany i może być wielokrotnie używany zamiast tworzenia go od nowa przy każdym uruchomieniu.

# Multimodal query engine

In [ ]:


gpt_4o = OpenAIMultiModal(model = CFG.model1, max_new_tokens=4096)

QA_PROMPT_TMPL = """\
Below we give parsed text from slides in two different formats, as well as the image.

We parse the text in both 'markdown' mode as well as 'raw text' mode. Markdown mode attempts \
to convert relevant diagrams into tables, whereas raw text tries to maintain the rough spatial \
layout of the text.

Use the image information first and foremost. ONLY use the text/markdown information
if you can't understand the image.

---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query. Explain whether you got the answer
from the parsed markdown or raw text or image, and if there's discrepancies, and your reasoning for the final answer.

Query: {query_str}
Answer: """

QA_PROMPT = PromptTemplate(QA_PROMPT_TMPL)

Ten kod konfiguruje komponenty do generowania odpowiedzi na pytania:

1. Konfiguracja modelu multimodalnego:
```python
gpt_4o = OpenAIMultiModal(model = CFG.model1, max_new_tokens=4096)
```
- Tworzy instancję modelu obsługującego tekst i obrazy
- Ustawia limit 4096 nowych tokenów w odpowiedzi

2. Definicja szablonu promptu (`QA_PROMPT_TMPL`):
- Informuje model, że otrzyma:
  - Tekst w dwóch formatach (Markdown i surowy tekst)
  - Powiązane obrazy
- Wyjaśnia różnice między formatami:
  - Markdown próbuje przekształcać diagramy w tabele
  - Surowy tekst zachowuje układ przestrzenny
- Instruuje model aby:
  - Priorytetowo traktował informacje z obrazów
  - Używał tekstu tylko gdy obraz jest niejasny
  - Wyjaśnił źródło informacji (Markdown/tekst/obraz)
  - Wskazał ewentualne rozbieżności
  - Uzasadnił finalną odpowiedź

3. Tworzenie obiektu szablonu:
```python
QA_PROMPT = PromptTemplate(QA_PROMPT_TMPL)
```
Przekształca szablon tekstowy w obiekt, który będzie używany do generowania promptów.

In [ ]:
query_engine = MultimodalQueryEngine(
    retriever=index.as_retriever(similarity_top_k=9), multi_modal_llm=gpt_4o
)

Ten kod tworzy silnik do obsługi zapytań multimodalnych:

- Używa wcześniej zdefiniowanej klasy `MultimodalQueryEngine`
- Konfiguruje go z dwoma głównymi komponentami:
  1. Retriever (wyszukiwarka) z indeksu:
     - Parametr `similarity_top_k=9` oznacza, że będzie zwracać 9 najbardziej podobnych fragmentów do zapytania
  2. Model multimodalny GPT-4 (`gpt_4o`) do analizy tekstu i obrazów

Utworzony `query_engine` będzie:
- Wyszukiwać odpowiednie fragmenty dokumentu na podstawie zapytania
- Analizować powiązane obrazy
- Generować odpowiedzi uwzględniające zarówno tekst jak i zawartość wizualną

In [ ]:
base_nodes = get_nodes(docs_text)

Ta linia wykonuje uproszczone przetwarzanie dokumentów tekstowych (`docs_text`) na węzły.

W przeciwieństwie do wcześniej użytej funkcji `get_text_nodes`, funkcja `get_nodes`:
- Pomija powiązania z obrazami
- Nie uwzględnia znaczników Markdown
- Zachowuje oryginalny tekst w węźle zamiast w metadanych
- Kopiuje wszystkie metadane z dokumentu źródłowego do każdego węzła
- Dzieli tekst używając separatora "\n---\n"

Te podstawowe węzły mogą służyć jako alternatywne źródło danych dla zapytań, które nie wymagają analizy obrazów.

In [ ]:
base_index = VectorStoreIndex(base_nodes, embed_model=embed_model)

Ta linia tworzy nowy indeks wektorowy:
- Używa podstawowych węzłów tekstowych (`base_nodes`)
- Wykorzystuje wcześniej skonfigurowany model do tworzenia embedingów (`embed_model`)
- Nie zawiera informacji o obrazach ani znacznikach Markdown

Jest to uproszczona wersja wcześniejszego indeksu, która może być używana do szybszego wyszukiwania gdy nie potrzebujemy analizy wizualnej.

Ten dodatkowy indeks może być przydatny do:
- Prostych zapytań tekstowych
- Porównywania wyników z pełnym indeksem multimodalnym
- Sytuacji gdy priorytetem jest szybkość, a nie pełna analiza dokumentu

In [ ]:
base_query_engine = base_index.as_query_engine(llm=llm, similarity_top_k=9)

Ta linia tworzy podstawowy silnik zapytań:
- Używa wcześniej utworzonego indeksu (`base_index`)
- Konfiguruje go z podstawowym modelem językowym (`llm`)
- Ustawia parametr `similarity_top_k=9`, co oznacza że będzie zwracać 9 najbardziej podobnych fragmentów

Jest to prostsza wersja silnika zapytań w porównaniu do `query_engine`:
- Obsługuje tylko tekst (bez obrazów)
- Używa standardowego modelu językowego zamiast multimodalnego
- Jest szybszy w działaniu
- Nadaje się do prostszych zapytań niewymagających analizy wizualnej

# Multimodal agent

In [ ]:

vector_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="vector_tool",
    description=(
        "Useful for retrieving specific context from the data. Do NOT select if question asks for a summary of the data."
    ),
)
agent = FunctionCallingAgentWorker.from_tools(
    [vector_tool], llm=llm, verbose=True
).as_agent()

Ten kod tworzy system agenta z narzędziami do odpowiadania na pytania:

1. Tworzy narzędzie wyszukiwania (`vector_tool`):
- Używa silnika zapytań multimodalnych (`query_engine`)
- Nadaje mu nazwę "vector_tool"
- Określa opis wyjaśniający, że narzędzie jest przydatne do wyszukiwania konkretnych informacji, ale nie do tworzenia podsumowań

2. Tworzy agenta (`agent`):
- Inicjalizuje go z listą narzędzi (na razie tylko `vector_tool`)
- Używa wcześniej zdefiniowanego modelu językowego (`llm`)
- Włącza tryb szczegółowy (`verbose=True`), który będzie pokazywał proces podejmowania decyzji
- Konwertuje na interfejs agenta za pomocą `as_agent()`

Ten agent może samodzielnie decydować, kiedy użyć narzędzia wektorowego do znalezienia odpowiednich informacji w dokumentach.

In [ ]:
# define a similar agent for the baseline
base_vector_tool = QueryEngineTool.from_defaults(
    query_engine=base_query_engine,
    name="vector_tool",
    description=(
        "Useful for retrieving specific context from the data. Do NOT select if question asks for a summary of the data."
    ),
)
base_agent = FunctionCallingAgentWorker.from_tools(
    [base_vector_tool], llm=llm, verbose=True
).as_agent()

Ten fragment tworzy analogiczny system agenta dla podstawowej wersji wyszukiwania:

1. Tworzy narzędzie `base_vector_tool`:
- Oparte na podstawowym silniku zapytań (`base_query_engine`)
- Ma taką samą nazwę i opis jak poprzednie narzędzie
- Działa tylko na tekście, bez obsługi obrazów

2. Tworzy agenta `base_agent`:
- Inicjalizuje go z podstawowym narzędziem wektorowym
- Używa tego samego modelu językowego
- Ma włączony tryb szczegółowy

Ten drugi agent służy jako punkt odniesienia - pozwala porównać wyniki wyszukiwania z wersją multimodalną i ocenić, czy dodatkowe informacje z obrazów poprawiają jakość odpowiedzi.

# Test

In [ ]:
query = (
    "Where does the company have production?"
)

In [ ]:
base_response = base_agent.query(query)

Added user message to memory: Where does the company have production?
=== Calling Function ===
Calling function: vector_tool with args: {"input": "production locations of the company"}
=== Function Output ===
The company has production locations in the Lower 48 states, Alaska, Canada, Norway, Libya, Malaysia, China, Qatar, and the Asia Pacific region.
=== LLM Response ===
The company has production locations in the Lower 48 states, Alaska, Canada, Norway, Libya, Malaysia, China, Qatar, and the Asia Pacific region.


In [ ]:

response = agent.query(query)

Added user message to memory: Where does the company have production?
=== Calling Function ===
Calling function: vector_tool with args: {"input": "production locations of the company"}
=== Function Output ===
The production locations of ConocoPhillips, as seen in the images, include:

- **Lower 48 (U.S.)**: Permian, Eagle Ford, Bakken
- **Alaska**
- **Canada**: Montney, Surmont
- **EMENA**: Norway, Libya, Qatar
- **Asia Pacific**: APLNG, Malaysia, China

This information was derived from the images provided, which show maps and lists of production areas. The parsed text also supports this information, listing similar locations. There are no discrepancies between the image and the parsed text regarding the production locations.
=== LLM Response ===
The company has production in the following locations:

- **Lower 48 (U.S.)**: Permian, Eagle Ford, Bakken
- **Alaska**
- **Canada**: Montney, Surmont
- **EMENA**: Norway, Libya, Qatar
- **Asia Pacific**: APLNG, Malaysia, China


In [ ]:
ppr(str(base_response))

('The company has production locations in the Lower 48 states, Alaska, Canada, '
 'Norway, Libya, Malaysia, China, Qatar, and the Asia Pacific region.')


In [ ]:
ppr(str(response))

('The company has production in the following locations:\n'
 '\n'
 '- **Lower 48 (U.S.)**: Permian, Eagle Ford, Bakken\n'
 '- **Alaska**\n'
 '- **Canada**: Montney, Surmont\n'
 '- **EMENA**: Norway, Libya, Qatar\n'
 '- **Asia Pacific**: APLNG, Malaysia, China')


In [ ]:
ppr(base_response.source_nodes[7].get_content(metadata_mode="all"))


('Secondary Investment Criteria Reinforce Resilient; Durable Returns\n'
 '                                Investment '
 'Criteria                               Balanced, Diversified, Disciplined '
 'Production Growth\n'
 '                                                                                   '
 'Production Mixl\n'
 '    '
 '8                                                                              '
 'Oil ~55%      NGL ~15%       North American Gas ~15%    International Gas ~ '
 '15%\n'
 '                                                                                                                    '
 'Production (MBOED)\n'
 '    [                   Disciplined                               Cost '
 'of                     ~4-5%\n'
 '                  Reinvestment Rate                               '
 'Supply        2,500\n'
 '                    Returns of capital                   Returns on '
 'capital              ProductioncXer                                  L

In [ ]:
ppr(response.source_nodes[7].get_content(metadata_mode="all"))


('page_num: 15\n'
 'image_path: data_images/affd3516-6466-4de8-a372-fbc4f7d6342b-page_3.jpg\n'
 'parsed_text_markdown: # Strong Track Record of Active Portfolio Management\n'
 '\n'
 '| Year | Activity |\n'
 '|------|----------|\n'
 '| 2016 | Production: 1.6 MMBOED <br> Resource < $40/BBL WTI: ~10 BBOE <br> '
 'Average Cost of Supply: < $40/BBL WTI <br> Resource Life: >18 years <br> '
 'Emissions Intensity: ~39 kg CO₂e/BOE |\n'
 '| 2017 | San Juan Exit <br> Canada Cenovus Transaction |\n'
 '| 2018 | WNS and GKA Working Interest Consolidations |\n'
 '| 2019 | U.K. Exit |\n'
 '| 2020 | Montney Acreage Acquisition |\n'
 '| 2021 | Concho and Shell Permian Acquisitions <br> Niobrara and '
 'Australia-West Exits |\n'
 '| 2022 | APLNG Acquisition <br> Indonesia Exit <br> Production: 1.7 MMBOED '
 '<br> Resource < $40/BBL WTI: ~20 BBOE <br> Average Cost of Supply: ~ $32/BBL '
 'WTI <br> Resource Life: >30 years <br> Emissions Intensity: ~22 kg CO₂e/BOE '
 '|\n'
 '\n'
 '**Cost of Supply Framewor